In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
import time
import torch.nn.init as ini
import multiprocessing
from multiprocessing import Queue
import random

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.local_conv1 = nn.Conv2d(4, 64, 7)
        self.bn1 = nn.BatchNorm2d(64)
        self.local_conv2 = nn.Conv2d(64, 64, 3)
        self.bn2 = nn.BatchNorm2d(64)
        self.local_conv3 = nn.Conv2d(4, 160, 13)
        self.bn3 = nn.BatchNorm2d(160)
        self.total_conv = nn.Conv2d(224, 5, 21)

    def forward(self, x):
        under_x = F.relu(self.bn3(self.local_conv3(x)))
        x = self.local_conv1(x)
        x = self.bn1(x)
        x = F.max_pool2d(F.relu(x), 4, stride = 1)
        x = self.local_conv2(x)
        x = self.bn2(x)
        x = F.max_pool2d(F.relu(x), 2, stride = 1)
        x = torch.cat((x, under_x), 1)
        x = self.total_conv(x)
        x = x.view(-1,5)
        return x
        
net = TwoPathConv()
net = net.cuda(1)
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
x = x.cuda(1)
y_pred = net.forward(x)
print(y_pred)

TwoPathConv (
  (local_conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (local_conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (local_conv3): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
  (bn3): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True)
  (total_conv): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
-0.0831 -0.6361 -0.0425  0.5718 -0.3551
[torch.cuda.FloatTensor of size 1x5 (GPU 1)]



In [ ]:
file_list = open('/home/yiqin/trainval-balanced.txt','r')
f = h5py.File('/home/yiqin/train.h5','r')
list1 = []
str1 = file_list.readlines()
for i in range(len(str1)):
    list1.append(str1[i][0:-1].split(' '))
print(len(list1))

In [ ]:
'''import pickle as pkl
output1 = open('HG0001_Val_list_unbalanced.pkl', 'wb')
output2 = open('training_list_unbalanced.pkl', 'wb')
pkl.dump(val_list, output1)
pkl.dump(train_list, output2)'''

In [ ]:
'''
val_list, train_list = create_train_val_brain(list1)
print(len(val_list))
print(len(train_list))
'''
import pickle as pkl
input1 = open('HG0001_Val_list.pkl', 'rb')
input2 = open('training_list.pkl', 'rb')
val_list = pkl.load(input1)
train_list = pkl.load(input2)
print(len(val_list))


In [ ]:
'''import pickle as pkl
output1 = open('HG0001_Val_list.pkl', 'wb')
output2 = open('training_list.pkl', 'wb')
pkl.dump(val_list, output1)
pkl.dump(train_list, output2)'''


prev_time = time.clock()
training_patch, training_label = create_sub_patch_phase1(512, 0)
print(time.clock() - prev_time)

In [ ]:
#weight init
for param in net.parameters():
    if len(param.size())==4:
        ini.uniform(param, a=-5e-3, b=5e-3)

In [ ]:
#without multiprocessing
def create_sub_patch_phase1(size, key):
    training_patch = []
    training_label = []
    len_data = len(train_list)
    for i in range(size):
        case,x,y,z,l = train_list[key * size + i]
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        content = f[case1][case2]
        img_patch = content[:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        training_patch.append(img_patch)
        training_label.append(l)
    train_patch = torch.from_numpy(np.array(training_patch))
    train_label = torch.from_numpy(np.array(training_label))
    return train_patch, train_label


def create_test_patch(img = 0, x = 16, z= 100):
    case = SAMPLE[img]
    case1 = case[:2]
    case2 = case[3:]
    batch = []
    _, X, Y, Z = f[case1][case2].shape
    for y in range(16, Y - 17):
        patch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
    patch = torch.from_numpy(np.array(patch))
    return patch


def create_val_patch(size):
    val_patch = []
    val_label = []
    len_data = len(val_list)
    for i in range(size):
        case,x,y,z,l = train_list[i]
        #print(i, key)
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        content = f[case1][case2]
        img_patch = content[:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        val_patch.append(img_patch)
        val_label.append(l)
    val_patch = torch.from_numpy(np.array(val_patch))
    val_label = torch.from_numpy(np.array(val_label))
    return val_patch, val_label

In [ ]:
prev_time = time.clock()
num_epoch = 2
batch_size = 256
iteration = len(train_list) // batch_size
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
net = TwoPathConv()
net = net.cuda(3)

#set hyperparams
learning_rate = 1e-4
l1_reg = 5e-5
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum = 0.9, weight_decay = 5e-5)

#create val set
val_patch_size = 256
val_x, val_y = create_val_patch(val_patch_size)
val_x, val_y = Variable(val_x.cuda(3)), val_y.cuda(3)

test_X = create_test_patch(img = 10)

for i in range(num_epoch):
    for j in range(iteration):
        training_patch, training_label = create_sub_patch_phase1(batch_size, j)
        x_train, y_train = Variable(training_patch.cuda(3)), Variable(training_label.cuda(3), requires_grad=False)
        #train
        y_pred = net.forward(x_train)
        y_pred = y_pred.view(-1,5)
        loss = F.cross_entropy(y_pred, y_train)#cross entropy loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #check accuracy
        if j%10 == 0:
            print('iteration %d /%d:'%(j, iteration), loss.data)
            print(float(j)/iteration,  'finished')
            val_pred = net.forward(val_x)
            val_pred = val_pred.view(-1, 5)
            _, predicted = torch.max(val_pred.data, 1)
            correct = (predicted == val_y).sum()
            print('Validation accuracy:', float(correct) / val_patch_size)
            print('time used:%.3f'% (time.clock() - prev_time))
    scheduler.step()
    
print ("phase1: successfully trained!")
torch.save(net.state_dict(), "/home/yiqin/phase1_TwoPathConv_net.txt")
print ("phase1: successfully saved!")

In [39]:
import pickle as pkl
input1 = open('training_list_unbalanced.pkl','rb')
input2 = open('HG0001_Val_list_unbalanced.pkl', 'rb')
train_list_unbalanced = pkl.load(input1)
val_list_unbalanced = pkl.load(input2)
print(len(train_list))
f = h5py.File('/home/yiqin/train.h5','r')


23709634


In [54]:
def create_training_patch_phase2(size, key):
    training_patch = []
    training_label = []
    len_data = len(train_list_unbalanced)
    for i in range(size):
        case,x,y,z,l = train_list[key * size + i]
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        content = f[case1][case2]
        img_patch = content[:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        training_patch.append(img_patch)
        training_label.append(l)
    train_patch = torch.from_numpy(np.array(training_patch))
    train_label = torch.from_numpy(np.array(training_label))
    return train_patch, train_label

def create_val_patch(size):
    val_patch = []
    val_label = []
    len_data = len(val_list_unbalanced)
    for i in range(size):
        case,x,y,z,l = val_list_unbalanced[i]
        #print(i, key)
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        content = f[case1][case2]
        img_patch = content[:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        val_patch.append(img_patch)
        val_label.append(l)
    val_patch = torch.from_numpy(np.array(val_patch))
    val_label = torch.from_numpy(np.array(val_label))
    return val_patch, val_label

In [41]:
num_epoch = 2
batch_size = 256
iteration = len(train_list_unbalanced) // batch_size
net = TwoPathConv()
net.load_state_dict(torch.load('/home/yiqin/phase1_TwoPathConv_net.txt'))
net = net.cuda(3)


learning_rate = 5e-3
l1_reg = 5e-5
optimizer = torch.optim.SGD(net.total_conv.parameters(), lr=learning_rate, momentum = 0.9, weight_decay = 5e-7)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

In [53]:
val_patch_size = 512
val_x, val_y = create_val_patch(val_patch_size)
print((val_y == 4).sum() *1.0 / 512)
val_x, val_y = Variable(val_x.cuda(3)), val_y.cuda(3)


0.02734375


In [43]:
val_patch_size = 512
val_x, val_y = create_val_patch(val_patch_size)
val_x, val_y = Variable(val_x.cuda(3)), val_y.cuda(3)

#test_X = create_test_patch(img = 10)

prev_time = time.clock()
for i in range(num_epoch):
    for j in range(iteration):
        training_patch, training_label = create_training_patch_phase2(batch_size, j)
        x_train, y_train = Variable(training_patch.cuda(3)), Variable(training_label.cuda(3), requires_grad=False)
        #train
        y_pred = net.forward(x_train)
        y_pred = y_pred.view(-1,5)
        loss = F.cross_entropy(y_pred, y_train)#cross entropy loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #check accuracy
        if j%10 == 0:
            print('iteration %d /%d:'%(j, iteration), loss.data)
            print(float(j)/iteration,  'finished')
            val_pred = net.forward(val_x)
            val_pred = val_pred.view(-1, 5)
            _, predicted = torch.max(val_pred.data, 1)
            correct = (predicted == val_y).sum()
            print('Validation accuracy:', float(correct) / val_patch_size)
            print('time used:%.3f'% (time.clock() - prev_time))
    scheduler.step()

iteration 0 /92615: 
 3.2928
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.0 finished
Validation accuracy: 0.923828125
time used:0.625
iteration 10 /92615: 
 1.5544
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.00010797387032338174 finished
Validation accuracy: 0.919921875
time used:5.997
iteration 20 /92615: 
 0.7757
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.00021594774064676348 finished
Validation accuracy: 0.9296875
time used:11.255
iteration 30 /92615: 
 0.8641
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.0003239216109701452 finished
Validation accuracy: 0.958984375
time used:16.535
iteration 40 /92615: 
 0.6860
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.00043189548129352696 finished
Validation accuracy: 0.912109375
time used:21.824
iteration 50 /92615: 
 0.4720
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.0005398693516169087 finished
Validation accuracy: 0.947265625
time used:27.106
iteration 60 /92615: 
 0.6171
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.00064784322194

iteration 530 /92615: 
 0.1281
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.005722615127139233 finished
Validation accuracy: 0.9375
time used:282.800
iteration 540 /92615: 
 0.1051
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.005830588997462614 finished
Validation accuracy: 0.951171875
time used:288.093
iteration 550 /92615: 
1.00000e-02 *
  9.2629
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.0059385628677859955 finished
Validation accuracy: 0.98046875
time used:293.350
iteration 560 /92615: 
 0.1002
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.006046536738109377 finished
Validation accuracy: 0.9375
time used:298.667
iteration 570 /92615: 
 0.1341
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.006154510608432759 finished
Validation accuracy: 0.953125
time used:303.947
iteration 580 /92615: 
 0.1720
[torch.cuda.FloatTensor of size 1 (GPU 3)]

0.006262484478756141 finished
Validation accuracy: 0.953125
time used:309.227
iteration 590 /92615: 
 0.2229
[torch.cuda.FloatTensor of size 1 

KeyboardInterrupt: 